In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from rich import print
from aifeel.util import read_corpus
from aifeel.util.feature_extraction import extract_features, feature_to_vector
from aifeel.util.preprocess import preprocess_text

In [4]:
from joblib import load

# Load the saved model
loaded_model = load('export/model/TFIDF-Model-Daro/logistic_regression_model.pkl')
# load the saved feature extraction
vectorizer = '';

In [6]:
def classify_reviews(file_name, output_file_name = 'result_testing_challenge.txt'):
    # read challenge file
    reviews = read_corpus(file_name)
    # preprocess text
    clean_reviews = [preprocess_text(review) for review in reviews]
    # initailize feature extraction technique
    
    X = vectorizer.transform(clean_reviews)

    predictions = loaded_model.predict(X)
    print(predictions)

    with open(output_file_name, 'w') as output_file:
        for prediction in predictions:
            output_file.write(f"{prediction}\n")


    return clean_reviews
classify_reviews('testing-challeng')

ValueError: X has 297 features, but LogisticRegression is expecting 74983 features as input.